# Chess retrograde analysis
_Description_

Known Issues:
* Does not work with pawns on the board

Possible improvements:
* Wrap the initialization in functions to make rerunning the notebook easier

## Imports
Imports needed to run this notebook

In [ ]:
import chess
from math import factorial


## Initial board entry
This will be replaced by @NicoMillers board setup code. For now you can enter a fen.
You can read more about fens [here](https://www.chess.com/terms/fen-chess).

The current example fen `8/8/8/8/4k3/8/8/K6R w - - 0 1` describes the scenario lone rook.
Additionally, the first board object is created.


In [ ]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
board = chess.Board(fen)

## Setting up the piece list
The piece list contains all pieces on the board.
This list is created in a multiple step process.
1. Get the piece map from the chess library.
   The piece map is a dictionary containing positions as keys and symbols as values.
   Example: `{28: Piece.from_symbol('k')}`
2. Iterate over the piece map and add the piece object to a new list.

The final piece list looks like this:
`[Piece.from_symbol('k')]`

In [ ]:
#get the piece map from the library
tmp_piece_map = board.piece_map()
pieceList = []
#iterate over the map and append the pieces to the list
for location in tmp_piece_map:
    pieceList.append(tmp_piece_map[location])

print(pieceList)

## Setting up the boardList
The board list is a list object containing `64! / (64-COUNT(Pieces))!` empty boards to be filled later.
This allows for the first piece in the piece list to be positioned on all 64 squares, the second piece on all remaining 63 (64\*63) the third on the remaining 62 (64\*63\*63) et cetera.

This is achieved by creating a new board and removing all default pieces.
Then the program iterates through a loop `64! / (64-COUNT(Pieces))!` times. On each iteration a copy of the empty board is added to the list.

In [ ]:
tmp_board = chess.Board()
tmp_board.clear()
board_list = []
list_length = factorial(64) // factorial(64 - len(pieceList))

for i in range(list_length):
    board_list.append(tmp_board.copy())

## The recursive placePieces helper function

Function arguments:
* boardList: A list containing empty or partially filled board objects
* pieceList: A list containing all pieces which should end up on the board
* pieceIndexStart: Index of the next piece to be placed
* start: range start for where to begin placing pieces in the boardList
* stop: range end for where to end placing pieces in the boardList

Function result:
This function recursively places the pieces in the pieceList with an index >= pieceIndexStart on the boards in the boardList.
The boards in the boardList will be filled in all possible combinations of the pieces.

Sideeffects:
This function modifies the supplied boardList.

Algorithm:
* Calculate piece offset
  The offset describes every how many boards the current piece needs to be placed on the next square.
  Example: if there are two pieces in the piece list, the first piece must be on all 64 squares, and for every position the first piece takes the second piece needs to be placed on the remaining 63 squares.
  ```
  O|     O|X  O|  O|
  --- -> ---  --- ---
   |      |   X|   |X

   |O    X|O   |O  |O
  --- -> ---  --- ---
   |      |   X|   |X
  ```
  In this 2x2 board the second piece has three possible positions, after the first piece was placed.
  So every fourth board the first piece needs to be placed on the next square.
  This can be scaled up to a 64x64 chess board.
  The offset in this example is 3.
  For larger boards the offset can be calculated with the following formula.
  `offset = (stop - start) // (64 - (len(pieceList) - 1) + pieceIndexStart)`
* Iterate over the board list, starting at *start*
    * Place the piece from the pieceList at the position pieceIndexStart on the first (empty) square.
    * If the offset is reached:
        * If there are pieces remaining recursively call the function with an incremented pieceIndexStart and new start and stop values. The new range is the area the current piece was placed.
        * Increment the square.
    * If the square is empty place the current piece on the square.



//TODO: Comments

In [ ]:
def placePieces(boardList, pieceList, pieceIndexStart, start, stop):
    squareIndex = -1 #first Run increments to 0
    offset = (stop - start) // (64 - (len(pieceList) - 1) + pieceIndexStart)
    for i in range(start, stop):
        if i % offset == 0:
            if pieceIndexStart < (len(pieceList)-1) and i - offset > 0:
                placePieces(boardList, pieceList, pieceIndexStart + 1, i - offset, i)
            squareIndex = squareIndex + 1
        square = chess.Square(squareIndex)
        if boardList[i].piece_at(square) is None:
            boardList[i].set_piece_at(square, pieceList[pieceIndexStart])

## Create all possible combinations of pieces on the board

In [ ]:
placePieces(board_list, pieceList, 0, 0, len(board_list))

## Check boards for validity and filter out invalid ones
Iterate over all previously created boards and check the status through the library. If it's valid add it to a new list.
Toggle the color whose turn it is and do the same.
Afterwards the list S contains all valid combinations.

In [ ]:
S = []
for board in board_list:
    #Board with white having the next turn
    board.turn = chess.WHITE
    status = board.status()
    if status == chess.Status.VALID or status == chess.Status.OPPOSITE_CHECK:
        S.append(board)
    boardBlackMove = board.copy()
    boardBlackMove.turn = chess.BLACK
    status = boardBlackMove.status()
    #The same board with black having the next turn
    if status == chess.Status.VALID or status == chess.Status.OPPOSITE_CHECK:
        S.append(boardBlackMove)


## Find won boards and add them to S_0
* Iterate over all boards in S.
* Check the status information provided by the library if there is a winner.
* If there is, add the board to the list S_0
  Additionally add them to S_0_ASCII for easier comparison in the next step.
* Else add the board to a temporary S (this is more efficient than removing the board from S)
* Replace S with the temporary S

In [ ]:
S_tmp = []
S_ASCII = []
S_0 = []
S_0_ASCII = []

for board in S:
    outcome = board.outcome()
    # If True Game has finished in a certain way
    if outcome != None:
        # A winner has been determined
        if outcome.winner is not None and board.is_valid() != False:
            S_0.append(board)
            S_0_ASCII.append((board.turn,board.__str__()))
    # Every other board
    else:
        S_tmp.append(board)
        S_ASCII.append((board.turn, board.__str__()))
S = S_tmp



## OneStepAway Function
This function returns (given two lists of boards) all boards from the second list, that are one move away from one board from the first list.
Additionally, it returns the second list, without the boards in the new list.

Function parameters:
* S_n: The list of boards n steps away from winning
* S: The list of all boards to be checked against

Sideeffects:
None.
All supplied lists stay the same, modified lists are returned in a tuple.

Algorithm:
* Store the ascii representations of all boards in S to make later comparisons easier
* Iterate over the boards in S_n
  * Iterate over the pseudoLegalMoves provided by the library.
    This list contains all possible moves, even from a winning state.
    __Important: This does not work with pawns - This needs to be fixed later__
    * Execute the move
    * Check if the board is in S (using S_ascii)
    * If true, append it to s_n1_ascii (as comparisons are done using the ascii representation)
* Iterate over S
  * If the board is in s_n1_ascii, add it to s_n1
  * Otherwise add it to s_tmp
* Return s_n1 and s_tmp

In [ ]:
def one_step_away(S_n, S):
    #variables
    s_n1 = []
    s_n1_ascii = []
    s_ascii = []
    s_tmp = []

    #create temporary list for comparison
    for chessBoard in S:
        s_ascii.append((chessBoard.turn, chessBoard.__str__))

    for chessBoard in S_n:
        for pLMove in chessBoard.pseudo_legal_moves:
            chessBoard.push(pLMove)
            if (chessBoard.turn, chessBoard.__str__()) in s_ascii:
                s_n1_ascii.append((chessBoard.turn, chessBoard.__str__()))
            chessBoard.pop()


    #create return lists
    for chessBoard in S:
        if(chessBoard.turn, chessBoard.__str__()) in s_n1_ascii:
            s_n1.append(chessBoard)
        else:
            s_tmp.append(chessBoard)

    return s_n1, s_tmp

## Find all boards one step away from winning
Validation needed

In [ ]:
S_1_cmp, S_cmp = one_step_away(S_0, S)

S_1 = []
S_1_ASCII = []
counter = 0
# print("WHITE: " + str(chess.WHITE))
# print("BLACK: " + str(chess.BLACK))
for board in S_0:
    # print("Round: " + str(counter))
    # print(board.__str__())
    # print("Turn: " + str(board.turn))
    board.turn = board.turn ^ True
    for move in board.pseudo_legal_moves:
        board.push(move)
        if ((board.turn, board.__str__())) in S_ASCII and board.is_valid():
            S_1_ASCII.append((board.turn, board.__str__()))
        board.pop()
    counter += 1

In [ ]:
S_0[0]

In [ ]:
S_tmp = []

for board in S:
    if((board.turn, board.__str__())) in S_1_ASCII:
        S_1.append(board)
    else:
        S_tmp.append(board)

S = S_tmp

In [ ]:
print(len(S_0))
for board in S_0:
    print(board.__str__())
    print(board.is_valid())
    print('\n')

In [ ]:
print(len(S_1))
for board in S_1:
    print(board.__str__())
    print(board.is_valid())
    print('\n')


In [ ]:
from collections import defaultdict

defaultdict 